In [1]:
import datetime as dt
import csv
from heapq import nlargest
import time

In [5]:
rdd1 = sc.textFile("hdfs://localhost:9000/user/emo/input/Reviews.csv")
rdd1 = rdd1.mapPartitions(lambda x: csv.reader(x))

In [3]:
def redFunc(line):
    list_ = []
    year = dt.datetime.fromtimestamp(float(line[7])).strftime('%Y')
    summary = line[8].split(" ")
    for word in summary:
        list_.append([year, word])
    return list_

In [6]:
startTime = time.time()

header = rdd1.first()
tmpRdd1 = rdd1.filter(lambda line: line != header) \
    .flatMap(redFunc) \
    .map(lambda couple: (tuple(couple), 1)) \
    .reduceByKey(lambda a,b:a+b) \
    .map(lambda line: (line[0][0], (line[0][1],line[1]))) \
    .groupByKey() \
    .flatMap(lambda line: (line[0], nlargest(10, line[1], lambda k: k[1])))
    
tmpRdd1.saveAsTextFile("hdfs://localhost:9000/user/emo/output/spark1/")
finishTime = time.time()
print(finishTime - startTime)

13.794430255889893


In [7]:
tmpRdd1.collect()

['2011',
 [('Great', 14036),
  ('the', 11614),
  ('for', 10993),
  ('and', 9355),
  ('a', 8065),
  ('but', 6501),
  ('Good', 6247),
  ('good', 5901),
  ('I', 5744),
  ('of', 5523)],
 '2012',
 [('Great', 16264),
  ('the', 13423),
  ('for', 12455),
  ('and', 11688),
  ('a', 9886),
  ('Good', 7993),
  ('I', 7487),
  ('but', 7333),
  ('good', 7313),
  ('to', 7100)],
 '2008',
 [('Great', 2974),
  ('the', 2611),
  ('for', 2234),
  ('and', 2116),
  ('a', 1702),
  ('Good', 1343),
  ('good', 1289),
  ('Best', 1227),
  ('to', 1216),
  ('but', 1110)],
 '2007',
 [('Great', 1969),
  ('the', 1544),
  ('for', 1331),
  ('and', 1293),
  ('a', 934),
  ('Good', 902),
  ('Best', 889),
  ('great', 844),
  ('good', 825),
  ('of', 797)],
 '2003',
 [('The', 10),
  ('the', 10),
  ('not', 9),
  ('and', 9),
  ('for', 8),
  ('A', 8),
  ('in', 7),
  ('but', 7),
  ('OF', 6),
  ('Excellent', 6)],
 '2001',
 [('a', 4),
  ('the', 4),
  ('Beetlejuice!', 4),
  ('movie,', 3),
  ('Beetlejuice', 3),
  ('is', 3),
  ('you', 3

In [5]:
sc.stop()